## 1. 알고리즘 설명

이 추천 시스템은 크게 두 가지 알고리즘을 혼합하여 사용합니다.

### 1.1. 콘텐츠 기반 필터링 (Content-Based Filtering)

-   **원리**: 사용자가 과거에 선호했던(혹은 현재 입력한) 아이템(영화)의 속성(장르)을 분석하여, 그와 유사한 속성을 가진 다른 아이템을 추천하는 방식입니다.
-   **핵심 기술 (TF-IDF 및 코사인 유사도)**:
    -   **TF-IDF (Term Frequency-Inverse Document Frequency)**: 영화의 '장르(genres)' 데이터를 벡터화하는 데 사용됩니다. 특정 장르(단어)가 한 영화(문서) 내에서 얼마나 자주 나타나는지(TF)와 전체 영화 데이터셋에서 얼마나 희귀하게 나타나는지(IDF)를 고려하여 장르의 중요도를 측정합니다. 이를 통해 각 영화를 장르 특징을 나타내는 벡터로 변환합니다.
    -   **코사인 유사도 (Cosine Similarity)**: TF-IDF로 벡터화된 영화들 간의 유사도를 측정하는 데 사용됩니다. 두 영화 벡터 간의 코사인 값(각도)을 계산하여, 각도가 작을수록(코사인 값이 1에 가까울수록) 두 영화의 장르가 비슷하다고 판단하고, 유사한 영화로 간주합니다.

### 1.2. 인기/고평점 영화 추천 (Popular/High-Rated Recommendations)

-   **원리**: 단순히 많은 사용자에게 인기가 많거나 평균 평점이 높은 영화를 추천하는 방식입니다.
-   **활용**: 콘텐츠 기반 추천만으로는 새로운 사용자나 취향이 뚜렷하지 않은 사용자에게 적합하지 않을 수 있으며, 추천 다양성이 부족할 수 있습니다. 이를 보완하기 위해 최소 평점 수(num_ratings)를 기준으로 하여 평균 평점(avg_rating)이 높은 영화를 우선적으로 추천 목록에 포함시킵니다. 이는 '탐색(Exploration)'과 '활용(Exploitation)'의 균형을 맞추는 역할을 합니다.

### 1.3. 혼합 추천 전략

이 시스템은 다음 순서로 추천 목록을 구성합니다.

1.  사용자가 지정한 `num_guaranteed_popular`만큼 **리뷰가 많고 평점이 높은 인기 영화**를 무조건 추천 목록에 포함합니다. (중복 및 사용자가 이미 입력한 영화는 제외)
2.  나머지 추천 개수를 채우기 위해, 사용자가 입력한 영화들을 기반으로 **콘텐츠 기반 유사 영화**를 찾아 추천합니다.
3.  만약 1, 2단계를 거치고도 최종 추천 개수(num_recommendations_total)가 부족하면, **추가적인 인기/고평점 영화**로 목록을 채웁니다.



## 2. 코드 작동 순서

### 2.1. 1단계: 라이브러리 임포트

-   `pandas`, `numpy`: 데이터 처리 및 조작을 위한 핵심 라이브러리입니다.
-   `sklearn.feature_extraction.text.TfidfVectorizer`: 텍스트 데이터를 TF-IDF 벡터로 변환합니다.
-   `sklearn.metrics.pairwise.linear_kernel`: 코사인 유사도(선형 커널)를 효율적으로 계산합니다.
-   `sklearn.model_selection.train_test_split`: 데이터셋을 훈련 세트와 테스트 세트로 분할합니다.
-   `sklearn.metrics.precision_score`, `recall_score`, `f1_score`: 추천 시스템 성능 평가 지표를 계산합니다.
-   `matplotlib.pyplot`, `seaborn`: 데이터 시각화를 위한 라이브러리입니다.
-   `pyspark.sql.SparkSession`, `pyspark.sql.functions`, `pyspark.sql.types`: Databricks 환경에서 Spark 데이터프레임 작업을 위한 라이브러리입니다.

### 2.2. 2단계: 데이터 로드

-   `SparkSession`을 초기화합니다. Databricks에서는 대부분 자동 초기화됩니다.
-   **Unity Catalog**의 `'1dt_team8_managed'.'movielens-small'.movies`와 `'1dt_team8_managed'.'movielens-small'.ratings` 테이블에서 영화 및 평점 데이터를 로드합니다.
-   Unity Catalog 로드에 실패하면, `/databricks-datasets/movielens/small/` 경로에 있는 CSV 파일에서 데이터를 로드하도록 폴백(fallback) 로직이 구현되어 있습니다.
-   콘텐츠 기반 추천을 위해 `movies_df_spark`와 `ratings_df_spark` (Spark DataFrame)를 `toPandas()`를 사용하여 `movies_df`와 `ratings_df` (Pandas DataFrame)로 변환합니다. 이는 TF-IDF 및 코사인 유사도 계산이 Pandas DataFrame에서 더 효율적이기 때문입니다.

### 2.3. 3단계: 데이터 전처리 및 콘텐츠 기반 추천 시스템 구축

-   **장르 데이터 클리닝**: `movies_df`의 `genres` 컬럼에 있는 누락된 값(NaN)을 빈 문자열('')로 채웁니다.
-   **TF-IDF 벡터화**: `TfidfVectorizer`를 사용하여 `movies_df['genres']` 컬럼의 텍스트 데이터를 TF-IDF 행렬(`tfidf_matrix`)로 변환합니다. `stop_words='english'`를 사용하여 일반적인 영어 불용어(예: 'a', 'the')를 제거합니다.
-   **코사인 유사도 계산**: `linear_kernel` 함수를 사용하여 `tfidf_matrix`로부터 모든 영화 쌍 간의 코사인 유사도를 계산하여 `cosine_sim` 행렬을 생성합니다. 이 행렬의 각 셀 `(i, j)`는 `i`번째 영화와 `j`번째 영화 간의 유사도를 나타냅니다.
-   **영화 인덱스 매핑**: 영화 제목을 기준으로 영화의 인덱스를 빠르게 찾을 수 있도록 `indices` Pandas Series를 생성합니다.
-   **헬퍼 함수 정의**:
    -   `get_title_from_id(movie_id)`: 영화 ID를 입력받아 해당 영화의 제목을 반환합니다.
    -   `get_movie_ids_from_names(movie_names)`: 영화 이름 리스트를 입력받아 해당 영화들의 ID 리스트를 반환하고, 찾을 수 없는 영화 이름도 반환합니다.

### 2.4. 4단계: 추천 함수 정의

-   `get_content_based_recommendations(movie_id, cosine_sim_matrix, movies_df_local, indices_local, num_recommendations)`:
    -   특정 `movie_id`를 기반으로 `cosine_sim_matrix`를 사용하여 가장 유사한 `num_recommendations`개의 영화를 찾아 Pandas DataFrame 형태로 반환합니다.
    -   입력 영화 자신은 추천 목록에서 제외합니다.
-   `recommend_movies_by_input(user_movie_ids, num_popular_or_high_rated_fixed, num_recommendations_total)`:
    -   사용자가 입력한 `user_movie_ids`를 기반으로 최종 `num_recommendations_total`개의 영화를 추천하는 메인 함수입니다.
    -   내부적으로 Spark DataFrame `ratings_df_spark`를 사용하여 영화별 `num_ratings`(리뷰 수)와 `avg_rating`(평균 평점)을 계산합니다.
    -   `min_ratings_threshold` (기본값 50) 이상의 리뷰를 가진 영화 중에서 평점이 높은 순서로 `num_popular_or_high_rated_fixed` 개수만큼의 인기 영화를 먼저 추가합니다.
    -   남은 추천 개수를 채우기 위해, 입력된 `user_movie_ids` 각각에 대해 `get_content_based_recommendations`를 호출하여 콘텐츠 기반 유사 영화를 가져와 추가합니다.
    -   마지막으로, 최종 추천 목록의 개수가 `num_recommendations_total`에 미달하면, 추가적인 인기/고평점 영화로 목록을 채웁니다.
    -   모든 추천이 완료되면, 추천 목록에서 사용자가 이미 입력한 영화와 중복된 영화를 제거하고, 최종 목록을 Spark DataFrame으로 변환하여 반환합니다. 이는 Databricks `display` 함수와의 호환성을 높입니다.

### 2.5. 5단계: 모델 테스트 (사용자 입력)

-   사용자에게 추천받고 싶은 영화의 이름 또는 ID를 3~5개 쉼표로 구분하여 입력하도록 요청합니다.
-   입력된 값이 숫자로만 구성되어 있으면 ID로, 그렇지 않으면 이름으로 간주합니다.
-   입력된 영화 이름/ID가 시스템에 존재하는지 확인하고, 유효한 입력이 들어올 때까지 반복하여 입력을 받습니다.
-   `recommend_movies_by_input` 함수를 호출하여 `num_recommendations_output` (총 추천 개수)와 `num_guaranteed_popular` (필수 포함 인기 영화 개수)를 조절하여 영화를 추천받습니다.
-   최종 추천된 영화 목록(제목과 장르)을 Databricks의 `display` 함수를 사용하여 출력합니다.

### 2.6. 6단계: 모델 성능 평가 및 시각화 (주석 처리됨)

-   `evaluate_recommender` 함수는 추천 시스템의 성능을 대략적으로 측정하기 위한 함수입니다.
    -   `ratings_df`를 훈련 세트(`train_ratings`)와 테스트 세트(`test_ratings`)로 분할합니다.
    -   테스트 세트에서 임의의 사용자(`sample_users`)를 샘플링합니다.
    -   각 샘플링된 사용자에 대해, 훈련 데이터에서 높은 평점을 준 영화들을 '입력 영화'로 가정하고 추천을 수행합니다.
    -   테스트 데이터에서 해당 사용자가 실제로 높은 평점을 준 영화(`actual_relevant_movie_ids`)와 추천된 영화(`recommended_movie_ids`)를 비교하여 **Precision (정밀도)**, **Recall (재현율)**, **F1-Score**를 계산합니다.
    -   이 평가 지표들은 추천 시스템이 얼마나 정확하고 포괄적으로 관련 영화를 추천하는지를 나타냅니다.
-   `model_performance_evaluation()` 함수는 `evaluate_recommender`를 호출하고, 계산된 평균 Precision, Recall, F1-Score를 출력합니다.
-   `matplotlib`과 `seaborn`을 사용하여 이러한 성능 지표를 막대 그래프로 시각화합니다.
-   **참고**: 현재 코드에서는 `model_performance_evaluation()` 함수가 주석 처리되어 있어 실행되지 않습니다. 평가를 원한다면 주석을 해제해야 합니다.


In [0]:
# Microsoft Azure Databricks Notebook.
# Content-Based + 인기 영화 추천 시스템 (Hybrid)

# 1. 라이브러리 임포트
import pandas as pd # 데이터 처리 및 조작을 위한 필수 라이브러리
import numpy as np # 수치 계산을 위한 라이브러리 (pandas와 함께 사용)
from sklearn.feature_extraction.text import TfidfVectorizer # 텍스트 데이터를 TF-IDF 특징 벡터로 변환
from sklearn.metrics.pairwise import linear_kernel # 코사인 유사도(선형 커널)를 효율적으로 계산
from sklearn.model_selection import train_test_split # 데이터셋을 훈련 세트와 테스트 세트로 분할
from sklearn.metrics import precision_score, recall_score, f1_score # 추천 시스템 성능 평가 지표 (정밀도, 재현율, F1-점수)
import matplotlib.pyplot as plt # 데이터 시각화를 위한 라이브러리
import seaborn as sns # matplotlib 기반의 통계 데이터 시각화 라이브러리
from pyspark.sql import SparkSession # Spark 애플리케이션의 진입점 (Spark DataFrame 사용에 필수)
from pyspark.sql.functions import col, lit, count, avg, desc, udf # Spark DataFrame 컬럼 조작 함수 및 사용자 정의 함수(UDF)
from pyspark.sql.types import ArrayType, StringType, IntegerType # Spark DataFrame 스키마 정의를 위한 데이터 타입

# Spark 세션 초기화 (Databricks에서는 자동으로 초기화되지만 명시적으로 정의 가능)
# getOrCreate()는 이미 세션이 존재하면 기존 세션을 반환하고, 없으면 새로 생성합니다.
spark = SparkSession.builder.appName("MovieRecommendation").getOrCreate()

# 2. 데이터 로드 (Unity Catalog 테이블에서 불러오도록 변경)
# '1dt_team8_managed.movielens-small' 데이터베이스에 movies, ratings 테이블이 있다고 가정합니다.
try:
    # Unity Catalog에서 movies 및 ratings 테이블을 Spark DataFrame으로 직접 읽어옵니다.
    # 이 방식은 Databricks 환경에서 최신 데이터 거버넌스 및 접근 방식을 따릅니다.
    movies_df_spark = spark.read.table("`1dt_team8_databricks`.`movielens-small`.movies")
    ratings_df_spark = spark.read.table("`1dt_team8_databricks`.`movielens-small`.ratings")
    print("데이터를 Unity Catalog 테이블에서 성공적으로 불러왔습니다.")
except Exception as e:
    # Unity Catalog 로드 실패 시, 기존의 /databricks-datasets 경로에서 CSV 파일을 로드합니다.
    # 이는 개발 또는 테스트 환경에서 데이터 접근에 문제가 있을 경우를 대비한 폴백(fallback) 로직입니다.
    print(f"Unity Catalog 테이블 로드 중 오류 발생: {e}")
    print("기존 경로 변수 방식으로 시도합니다. (데이터 경로를 다시 확인해주세요)")
    movies_path = "/databricks-datasets/movielens/small/movies.csv"
    ratings_path = "/databricks-datasets/movielens/small/ratings.csv"
    movies_df_spark = spark.read.csv(movies_path, header=True, inferSchema=True) # CSV 파일 로드 (첫 줄은 헤더, 스키마 자동 추론)
    ratings_df_spark = spark.read.csv(ratings_path, header=True, inferSchema=True)

# Pandas DataFrame으로 변환 (콘텐츠 기반 추천을 위해 TF-IDF를 사용하기 위함)
# scikit-learn 라이브러리는 주로 Pandas DataFrame에서 작동하므로 Spark DataFrame을 변환합니다.
# 대규모 데이터셋의 경우 이 변환 대신 Spark MLlib (Spark의 기계 학습 라이브러리) 사용을 고려해야 합니다.
# Spark MLlib은 분산 처리를 통해 대용량 데이터에서도 TF-IDF 및 유사도 계산을 수행할 수 있습니다.
movies_df = movies_df_spark.toPandas()
ratings_df = ratings_df_spark.toPandas()

# 데이터 확인 (주석 처리됨)
# print("Movies DataFrame head:")
# print(movies_df.head()) # movies DataFrame의 첫 5행 출력
# print("\nRatings DataFrame head:")
# print(ratings_df.head()) # ratings DataFrame의 첫 5행 출력

# 3. 데이터 전처리 및 콘텐츠 기반 추천 시스템 구축

# 장르 데이터 클리닝 및 TF-IDF 벡터화
# genres 컬럼의 누락된 값 처리 (결측치는 빈 문자열로 대체하여 TF-IDF 변환 시 오류 방지)
movies_df['genres'] = movies_df['genres'].fillna('')

# TfidfVectorizer 객체 생성: 텍스트(장르 문자열)를 TF-IDF 특징 벡터로 변환합니다.
# stop_words='english'를 설정하여 영어 불용어(예: "and", "the")를 벡터화에서 제외합니다.
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
# fit_transform: genres 컬럼의 모든 영화 장르를 학습하고(fit), 이를 TF-IDF 행렬로 변환합니다(transform).
tfidf_matrix = tfidf_vectorizer.fit_transform(movies_df['genres'])

print(f"\nTF-IDF matrix shape: {tfidf_matrix.shape}") # 생성된 TF-IDF 행렬의 차원 출력 (영화 수, 고유 장르 특징 수)

# 코사인 유사도 계산 (콘텐츠 기반 유사도)
# linear_kernel: 두 벡터 세트의 내적을 계산하여 코사인 유사도를 효율적으로 구합니다.
# 여기서는 tfidf_matrix 자기 자신과의 유사도를 계산하여 모든 영화 쌍 간의 유사도 행렬을 만듭니다.
# 결과인 cosine_sim은 N x N 행렬이며, 여기서 N은 영화의 수입니다.
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# 영화 제목과 인덱스를 매핑하는 시리즈 생성
# 영화 제목을 입력받았을 때 해당 영화의 pandas DataFrame 인덱스를 빠르게 찾기 위해 사용됩니다.
# drop_duplicates()는 중복된 영화 제목이 있을 경우 첫 번째 인덱스만 유지합니다.
indices = pd.Series(movies_df.index, index=movies_df['title']).drop_duplicates()

# 영화 ID로 제목을 찾는 헬퍼 함수 (Pandas DataFrame 사용)
def get_title_from_id(movie_id):
    """
    주어진 영화 ID에 해당하는 영화 제목을 movies_df에서 찾아 반환합니다.
    """
    if movie_id in movies_df['movieId'].values:
        # movie_id가 일치하는 행을 찾아 'title' 컬럼의 첫 번째 값(iloc[0])을 반환
        return movies_df[movies_df['movieId'] == movie_id]['title'].iloc[0]
    return None # 해당 ID의 영화가 없으면 None 반환

# 영화 이름으로 ID를 찾는 헬퍼 함수 (Pandas DataFrame 사용)
def get_movie_ids_from_names(movie_names):
    """
    주어진 영화 이름 리스트에 해당하는 영화 ID들을 찾아 반환합니다.
    찾을 수 없는 영화 이름도 별도로 반환합니다.
    """
    found_ids = [] # 찾은 영화 ID들을 저장할 리스트
    not_found_names = [] # 찾지 못한 영화 이름들을 저장할 리스트
    for name in movie_names:
        # 정확히 일치하는 제목 찾기 (대소문자 구분 없이 비교)
        match = movies_df[movies_df['title'].str.lower() == name.lower()]
        if not match.empty:
            # 일치하는 영화가 있으면 해당 movieId를 추가
            found_ids.append(match['movieId'].iloc[0])
        else:
            # 일치하는 영화가 없으면 not_found_names에 추가
            not_found_names.append(name)
    return found_ids, not_found_names # 찾은 ID와 찾지 못한 이름 리스트 반환


# 4. 추천 함수 정의

def get_content_based_recommendations(movie_id, cosine_sim_matrix, movies_df_local, indices_local, num_recommendations=10):
    """
    콘텐츠(장르) 기반으로 유사한 영화를 추천합니다. (movie_id 기반)

    Args:
        movie_id (int): 기준이 될 영화의 ID.
        cosine_sim_matrix (np.array): 영화 간 코사인 유사도 행렬.
        movies_df_local (pd.DataFrame): 영화 정보가 담긴 Pandas DataFrame.
        indices_local (pd.Series): 영화 제목과 인덱스를 매핑하는 Series.
        num_recommendations (int): 추천할 영화의 개수.

    Returns:
        pd.DataFrame: 추천된 영화들의 정보 (Pandas DataFrame).
    """
    movie_title = get_title_from_id(movie_id) # 영화 ID로부터 제목을 가져옴
    # 영화 제목이 없거나, 인덱스 매핑에 없는 경우 빈 DataFrame 반환 (유효하지 않은 입력)
    if not movie_title or movie_title not in indices_local:
        return pd.DataFrame()

    idx = indices_local[movie_title] # 기준 영화의 인덱스를 가져옴
    # 해당 영화의 유사도 점수 리스트를 가져옴 (enumerate로 인덱스와 점수 쌍 생성)
    sim_scores = list(enumerate(cosine_sim_matrix[idx]))
    # 유사도 점수를 기준으로 내림차순 정렬
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 자기 자신 제외: 첫 번째 요소(자기 자신과의 유사도 1.0)를 제외하고 상위 N개 선택
    # num_recommendations + 1을 하는 이유는 자기 자신을 제외하고 원하는 개수를 얻기 위함
    sim_scores = sim_scores[1:num_recommendations + 1]
    # 유사도 점수 리스트에서 영화 인덱스만 추출
    movie_indices = [i[0] for i in sim_scores]
    # 해당 인덱스들의 영화 정보를 movies_df_local에서 가져와 반환
    return movies_df_local.iloc[movie_indices]


def recommend_movies_by_input(user_movie_ids, num_recommendations_total, num_popular_or_high_rated_fixed):
    """
    사용자가 입력한 영화 ID를 기반으로 콘텐츠 기반 추천 + 인기/고평점 영화를 섞어서 추천합니다.
    num_popular_or_high_rated_fixed 만큼 리뷰 많고 평점 높은 영화를 무조건 포함합니다.

    Args:
        user_movie_ids (list): 사용자가 입력한 영화 ID 리스트.
        num_recommendations_total (int): 최종적으로 추천할 총 영화 개수.
        num_popular_or_high_rated_fixed (int): 추천 목록에 무조건 포함할 인기/고평점 영화의 개수.

    Returns:
        pyspark.sql.DataFrame: 최종 추천된 영화들의 Spark DataFrame.
    """
    recommended_movies = pd.DataFrame() # 추천된 영화들을 저장할 Pandas DataFrame (임시)
    seen_movie_ids = set(user_movie_ids) # 사용자가 이미 입력한 영화 ID를 저장하여 추천 목록에서 제외

    # --- 1. 리뷰 많고 평점 높은 영화 목록 미리 계산 (Spark DataFrame 사용) ---
    # ratings_df_spark를 사용하여 영화별 리뷰 수(num_ratings)와 평균 평점(avg_rating)을 집계합니다.
    movie_stats_df_spark = ratings_df_spark.groupBy("movieId").agg(
        count(col("rating")).alias("num_ratings"), # 각 영화의 리뷰 개수
        avg(col("rating")).alias("avg_rating")     # 각 영화의 평균 평점
    )

    min_ratings_threshold = 50 # 최소 리뷰 수 임계값: 너무 적은 리뷰로 인한 평점 왜곡 방지
    # 최소 리뷰 수를 만족하는 영화들만 필터링하고, 평균 평점, 리뷰 수 내림차순으로 정렬합니다.
    # movies_df_spark와 조인하여 영화 제목 등의 정보를 가져옵니다.
    popular_movies_spark_filtered = movie_stats_df_spark.filter(col("num_ratings") >= min_ratings_threshold) \
                                                          .orderBy(desc("avg_rating"), desc("num_ratings")) \
                                                          .join(movies_df_spark, "movieId", "left")
    popular_movies_df = popular_movies_spark_filtered.toPandas() # Pandas DataFrame으로 변환

    # --- 2. 리뷰 많고 평점 높은 영화를 지정된 개수만큼 무조건 포함 ---
    popular_added_count = 0 # 현재까지 추가된 인기 영화 수
    for _, row in popular_movies_df.iterrows():
        if row['movieId'] not in seen_movie_ids: # 이미 추천 목록에 있거나 사용자가 입력한 영화는 제외
            # recommended_movies DataFrame에 현재 인기 영화 정보 추가
            recommended_movies = pd.concat([recommended_movies, pd.DataFrame([row])], ignore_index=True)
            seen_movie_ids.add(row['movieId']) # 중복 추가 방지를 위해 ID 기록
            popular_added_count += 1
            if popular_added_count >= num_popular_or_high_rated_fixed: # 필요한 인기 영화 개수를 채웠으면 중단
                break
    
    # --- 3. 나머지 추천 개수 (총 num_recommendations_total 중 인기영화 제외) ---
    remaining_recommendations_slots = num_recommendations_total - popular_added_count
    
    # 남은 슬롯이 없거나 음수가 되면 더 이상 콘텐츠 기반 추천을 할 필요 없음
    if remaining_recommendations_slots <= 0:
        # 최종 추천 목록에서 필요한 개수만큼 자르기 (중복 및 입력 영화 제거 후)
        final_recommendations = recommended_movies[~recommended_movies['movieId'].isin(user_movie_ids)] \
                                .drop_duplicates(subset=['movieId']) \
                                .head(num_recommendations_total) \
                                .reset_index(drop=True)
        return spark.createDataFrame(final_recommendations) # Spark DataFrame으로 변환하여 반환


    # --- 4. 콘텐츠 기반 추천으로 나머지 채우기 ---
    # 각 입력 영화에 대한 콘텐츠 기반 추천을 가져와 합치기
    for movie_id in user_movie_ids:
        # 필요한 remaining_recommendations_slots보다 더 많이 가져와서 중복 및 이미 추가된 인기영화 제거
        # (remaining_recommendations_slots * 2)는 나중에 중복 제거 후에도 충분한 개수를 확보하기 위함
        content_recs_for_one_movie = get_content_based_recommendations(
            movie_id, cosine_sim, movies_df, indices, remaining_recommendations_slots * 2
        )
        for _, row in content_recs_for_one_movie.iterrows():
            if row['movieId'] not in seen_movie_ids: # 이미 추천 목록에 있거나 사용자가 입력한 영화는 제외
                # recommended_movies DataFrame에 현재 콘텐츠 기반 추천 영화 정보 추가
                recommended_movies = pd.concat([recommended_movies, pd.DataFrame([row])], ignore_index=True)
                seen_movie_ids.add(row['movieId']) # 중복 추가 방지를 위해 ID 기록
                # 필요한 콘텐츠 기반 추천 개수(전체에서 인기 영화 개수 제외)를 채웠으면 중단
                if len(recommended_movies) - popular_added_count >= remaining_recommendations_slots:
                    break
        # 전체 콘텐츠 기반 추천 개수를 채웠으면 더 이상 다른 입력 영화를 보지 않아도 됨
        if len(recommended_movies) - popular_added_count >= remaining_recommendations_slots:
            break

    # --- 5. 여전히 추천 개수가 부족하면 추가 인기/고평점 영화로 채우기 ---
    # (총 추천 개수 - 현재까지 추천된 영화 개수) 만큼 추가 인기 영화가 필요할 경우
    if len(recommended_movies) < num_recommendations_total:
        additional_popular_needed = num_recommendations_total - len(recommended_movies)
        added_count = 0
        for _, row in popular_movies_df.iterrows():
            if row['movieId'] not in seen_movie_ids: # 이미 추천 목록에 있거나 사용자가 입력한 영화는 제외
                recommended_movies = pd.concat([recommended_movies, pd.DataFrame([row])], ignore_index=True)
                seen_movie_ids.add(row['movieId'])
                added_count += 1
                if added_count >= additional_popular_needed: # 필요한 추가 인기 영화 개수를 채웠으면 중단
                    break

    # 최종 추천 목록에서 필요한 개수만큼 자르기 (중복 및 입력 영화 제거 후)
    # 1. 사용자가 이미 본(입력한) 영화는 추천 목록에서 제외합니다.
    # 2. movieId를 기준으로 중복을 제거합니다.
    # 3. head()를 사용하여 최종적으로 num_recommendations_total 개수만큼의 영화만 선택합니다.
    # 4. reset_index(drop=True)로 인덱스를 재설정합니다.
    final_recommendations = recommended_movies[~recommended_movies['movieId'].isin(user_movie_ids)] \
                            .drop_duplicates(subset=['movieId']) \
                            .head(num_recommendations_total) \
                            .reset_index(drop=True)

    # Spark DataFrame으로 변환하여 반환 (Databricks display 함수 호환성을 위해)
    return spark.createDataFrame(final_recommendations)


# 5. 모델 테스트 (사용자 입력)

# 사용자에게 영화 이름 또는 ID 입력 요청
num_recommendations_output = 8 # 최종 추천할 영화 개수 설정
num_guaranteed_popular = 2 # 추천 목록에 포함할 인기 영화 개수 조절 (총 추천 개수 내에서)

while True: # 유효한 사용자 입력이 들어올 때까지 반복
    user_input_str = input("추천받고 싶은 영화 이름 또는 ID를 쉼표로 구분하여 3~5개 입력하세요 (예: Toy Story, Jumanji, Braveheart 또는 1,2,13): ")
    
    is_id_input = True # 입력이 ID인지 이름인지 판단하는 플래그
    processed_inputs = [] # 파싱된 입력을 저장할 리스트
    for item in user_input_str.split(','): # 쉼표로 분리하고 각 항목 공백 제거
        stripped_item = item.strip()
        try:
            processed_inputs.append(int(stripped_item)) # 숫자로 변환 시도 (ID인 경우)
        except ValueError:
            is_id_input = False # 숫자로 변환 실패 시 이름으로 간주
            processed_inputs.append(stripped_item) # 이름 그대로 추가

    if not (3 <= len(processed_inputs) <= 5): # 입력 개수 검증 (3개~5개)
        print("영화는 3개에서 5개 사이로 입력해야 합니다. 다시 입력해주세요.")
        continue # 다시 입력 요청

    input_movie_ids = [] # 실제 추천 시스템에 사용할 영화 ID 리스트
    input_movie_names_display = [] # 사용자에게 보여줄 영화 이름 리스트

    if is_id_input: # 입력이 영화 ID인 경우
        input_movie_ids = processed_inputs
        # Spark DataFrame에서 입력된 ID에 해당하는 영화 정보(ID, 제목)를 조회
        found_movie_info = movies_df_spark.filter(col("movieId").isin(input_movie_ids)).select("movieId", "title").collect()
        found_ids_set = {row.movieId for row in found_movie_info} # 찾은 영화 ID 집합
        
        if len(found_ids_set) != len(input_movie_ids): # 입력한 ID 중 찾을 수 없는 ID가 있는 경우
            missing_ids = set(input_movie_ids) - found_ids_set
            print(f"오류: 다음 영화 ID를 찾을 수 없습니다: {list(missing_ids)}. 다시 입력해주세요.")
            continue # 다시 입력 요청
        
        # 찾은 영화 ID를 제목과 매핑하여 사용자에게 보여줄 이름 리스트 생성
        movie_id_to_title = {row.movieId: row.title for row in found_movie_info}
        input_movie_names_display = [movie_id_to_title[mid] for mid in input_movie_ids]
        print(f"\n입력된 영화 ID: {input_movie_ids}")

    else: # 입력이 영화 이름인 경우
        input_movie_names = processed_inputs
        # Pandas DataFrame 기반의 get_movie_ids_from_names 함수 사용 (이름을 ID로 변환)
        input_movie_ids, not_found_names = get_movie_ids_from_names(input_movie_names)
        
        if not_found_names: # 찾을 수 없는 영화 이름이 있는 경우
            print(f"다음 영화를 찾을 수 없거나 모호하여 추천을 진행할 수 없습니다: {', '.join(not_found_names)}")
            continue # 다시 입력 요청
        elif len(input_movie_ids) != len(input_movie_names): # 찾았으나 개수가 다르면 문제 (일부만 찾은 경우)
            print("입력된 모든 영화 이름을 영화 ID로 변환하지 못했습니다. 다시 시도해주세요.")
            continue # 다시 입력 요청
        
        input_movie_names_display = input_movie_names # 사용자에게 보여줄 이름은 입력 그대로 사용

    break # 유효한 입력이 들어왔으므로 루프 종료

print(f"\n입력된 영화: {input_movie_names_display} (ID: {input_movie_ids})를 기반으로 영화를 추천합니다:")
# recommend_movies_by_input 함수는 Spark DataFrame을 반환
# num_guaranteed_popular 변수를 사용하여 최종 추천 목록에 포함될 인기 영화의 개수를 조절
recommended_df = recommend_movies_by_input(input_movie_ids, num_recommendations_output, num_guaranteed_popular)
# Databricks의 display 함수를 사용하여 결과를 테이블 형태로 시각화하여 출력
# 'title'과 'genres' 컬럼만 선택하여 보여줍니다.
display(recommended_df.select('title', 'genres'))


# 6. 모델 성능 평가 및 시각화

# 평가 지표 계산 함수 (간단한 추천 시스템의 Precision, Recall 계산)
def evaluate_recommender(model_func, movies_df_eval, ratings_df_eval, indices_eval, cosine_sim_matrix_eval,
                         test_ratings_df, num_recommendations_total=10, num_popular_fixed=1):
    """
    추천 시스템의 평균 Precision, Recall, F1-Score를 계산합니다.
    모든 사용자에 대해 평가하기는 어려우므로, 테스트 데이터셋에서 일부 사용자를 샘플링하여 평가합니다.

    Args:
        model_func (function): 영화 추천을 수행하는 함수 (여기서는 recommend_movies_by_input).
        movies_df_eval (pd.DataFrame): 영화 정보 DataFrame.
        ratings_df_eval (pd.DataFrame): 전체 평점 정보 DataFrame.
        indices_eval (pd.Series): 영화 제목과 인덱스 매핑 Series.
        cosine_sim_matrix_eval (np.array): 코사인 유사도 행렬.
        test_ratings_df (pd.DataFrame): 테스트용 평점 DataFrame.
        num_recommendations_total (int): 모델이 추천할 총 영화 개수.
        num_popular_fixed (int): 모델이 고정적으로 포함할 인기 영화 개수.

    Returns:
        tuple: (평균 Precision, 평균 Recall, 평균 F1-Score, 평가된 사용자 수)
    """
    total_precision = 0 # 전체 Precision 합계
    total_recall = 0    # 전체 Recall 합계
    total_f1 = 0        # 전체 F1-Score 합계
    num_users_evaluated = 0 # 평가된 사용자 수

    # 테스트 데이터셋에서 임의의 사용자 샘플링
    # 전체 사용자 중 최대 50명 또는 고유 사용자 수 중 더 작은 값을 샘플링합니다.
    sample_users = test_ratings_df['userId'].sample(min(50, test_ratings_df['userId'].nunique()), random_state=42).unique()

    for user_id in sample_users:
        # 해당 사용자의 훈련 데이터에서 가장 높은 평점을 준 영화들을 '입력 영화'로 가정
        # 사용자가 훈련 세트에서 평점을 매긴 영화들을 평점 내림차순으로 정렬합니다.
        user_train_movies = train_ratings[train_ratings['userId'] == user_id].sort_values(by='rating', ascending=False)
        
        # 훈련 데이터에서 최소 3개 이상의 영화를 본 사용자만 고려
        if len(user_train_movies) < 3:
            continue # 영화 수가 적으면 평가에서 제외
            
        # 가장 높은 평점을 준 상위 3개 영화의 ID를 시드(seed) 영화로 사용
        # 이 영화들을 사용자가 "좋아하는 영화"로 간주하고 추천을 시작합니다.
        seed_movie_ids = user_train_movies.head(3)['movieId'].tolist()

        if not seed_movie_ids: # 시드 영화 ID가 없으면 건너뜀
            continue

        # 추천 받기
        # model_func (recommend_movies_by_input) 함수를 호출하여 추천을 받습니다.
        # 이 함수는 Spark DataFrame을 반환하므로 .toPandas()로 변환하여 처리합니다.
        recommended_movies_df_spark = model_func(
            seed_movie_ids, num_recommendations_total, num_popular_or_high_rated_fixed=num_popular_fixed
        )
        recommended_movies_df = recommended_movies_df_spark.toPandas()
        
        if recommended_movies_df.empty: # 추천된 영화가 없으면 건너뜀
            continue

        # 추천된 영화 ID 목록 (집합 형태로 변환하여 빠른 조회 가능)
        recommended_movie_ids = set(recommended_movies_df['movieId'].tolist())

        # 해당 사용자가 테스트 데이터에서 실제로 높은 평점을 준 영화 목록 (예: 4점 이상)
        # 이 영화들은 사용자가 "실제로 관련 있다고 판단한" 영화들입니다.
        actual_relevant_movies_df = test_ratings_df[(test_ratings_df['userId'] == user_id) & (test_ratings_df['rating'] >= 4)]
        actual_relevant_movie_ids = set(actual_relevant_movies_df['movieId'].tolist())

        # 교집합 (추천된 영화 중 실제 관련성 있는 영화 = True Positives)
        true_positives = len(recommended_movie_ids.intersection(actual_relevant_movie_ids))

        # Precision: 추천된 영화 중 실제로 관련성 있는 영화의 비율
        # (True Positives) / (True Positives + False Positives)
        precision = true_positives / len(recommended_movie_ids) if len(recommended_movie_ids) > 0 else 0

        # Recall: 실제 관련성 있는 영화 중 추천된 영화의 비율
        # (True Positives) / (True Positives + False Negatives)
        recall = true_positives / len(actual_relevant_movie_ids) if len(actual_relevant_movie_ids) > 0 else 0

        # F1-score: Precision과 Recall의 조화 평균 (두 지표의 균형을 나타냄)
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

        total_precision += precision # 각 사용자의 Precision을 합산
        total_recall += recall       # 각 사용자의 Recall을 합산
        total_f1 += f1               # 각 사용자의 F1-Score를 합산
        num_users_evaluated += 1     # 평가된 사용자 수 증가

    if num_users_evaluated == 0:
        return 0, 0, 0, 0 # 평가된 사용자가 없으면 0 반환

    # 평가된 사용자 수로 나누어 평균값 계산
    avg_precision = total_precision / num_users_evaluated
    avg_recall = total_recall / num_users_evaluated
    avg_f1 = total_f1 / num_users_evaluated

    return avg_precision, avg_recall, avg_f1, num_users_evaluated

# 평가를 위한 데이터 분할 (사용자-영화 상호작용 데이터)
# ratings_df를 훈련(80%)과 테스트(20%) 세트로 분할합니다. random_state는 재현 가능성을 보장합니다.
train_ratings, test_ratings = train_test_split(ratings_df, test_size=0.2, random_state=42)

# 모델 성능 평가를 실행하는 함수
def model_performance_evaluation():
    print("\n모델 성능 평가를 시작합니다...")
    # evaluate_recommender 호출 시 num_popular_fixed 변수를 전달하여 인기 영화 개수 조절
    # 여기서는 사용자 입력 테스트에서 사용된 num_guaranteed_popular 값을 사용합니다.
    avg_precision, avg_recall, avg_f1, num_evaluated_users = evaluate_recommender(
        recommend_movies_by_input, movies_df, ratings_df, indices, cosine_sim, test_ratings,
        num_recommendations_total=num_recommendations_output, # 추천할 총 영화 개수
        num_popular_fixed=num_guaranteed_popular # 포함될 인기 영화 개수
    )

    print(f"\n평가된 사용자 수: {num_evaluated_users}")
    print(f"평균 Precision: {avg_precision:.4f}") # 소수점 4자리까지 출력
    print(f"평균 Recall: {avg_recall:.4f}")
    print(f"평균 F1-Score: {avg_f1:.4f}")

    # 성능 시각화
    metrics = ['Precision', 'Recall', 'F1-Score'] # 지표 이름 리스트
    values = [avg_precision, avg_recall, avg_f1] # 계산된 평균값 리스트

    plt.figure(figsize=(8, 6)) # 그래프 크기 설정
    sns.barplot(x=metrics, y=values, palette='viridis') # 막대 그래프 생성 (색상 팔레트 'viridis')
    plt.title('Recommendation System Performance Metrics') # 그래프 제목
    plt.ylabel('Score') # Y축 레이블
    plt.ylim(0, 1) # Y축 범위 (0에서 1까지)
    # 각 막대 위에 정확한 점수 값 표시
    for i, v in enumerate(values):
        plt.text(i, v + 0.02, f"{v:.2f}", ha='center', va='bottom', fontsize=10) # 텍스트 위치 및 포맷 설정
    plt.show() # 그래프 출력

# 모델 성능 평가 함수 호출 (현재 주석 처리됨)
model_performance_evaluation()

In [0]:
# Microsoft Azure Databricks Notebook.
# Content-Based + 인기 영화 추천 시스템 (Hybrid)

# 1. 라이브러리 임포트
import pandas as pd # 데이터 처리 및 조작을 위한 필수 라이브러리
import numpy as np # 수치 계산을 위한 라이브러리 (pandas와 함께 사용)
from sklearn.feature_extraction.text import TfidfVectorizer # 텍스트 데이터를 TF-IDF 특징 벡터로 변환
from sklearn.metrics.pairwise import linear_kernel # 코사인 유사도(선형 커널)를 효율적으로 계산
# from sklearn.model_selection import train_test_split # 데이터셋을 훈련 세트와 테스트 세트로 분할 (Spark DataFrame 사용으로 변경)
# from sklearn.metrics import precision_score, recall_score, f1_score # 추천 시스템 성능 평가 지표 (정밀도, 재현율, F1-점수) (직접 계산으로 변경)
import matplotlib.pyplot as plt # 데이터 시각화를 위한 라이브러리
import seaborn as sns # matplotlib 기반의 통계 데이터 시각화 라이브러리
from pyspark.sql import SparkSession # Spark 애플리케이션의 진입점 (Spark DataFrame 사용에 필수)
from pyspark.sql.functions import col, lit, count, avg, desc, udf # Spark DataFrame 컬럼 조작 함수 및 사용자 정의 함수(UDF)
from pyspark.sql.types import ArrayType, StringType, IntegerType, StructType, StructField # Spark DataFrame 스키마 정의를 위한 데이터 타입

# Spark 세션 초기화 (Databricks에서는 자동으로 초기화되지만 명시적으로 정의 가능)
spark = SparkSession.builder.appName("MovieRecommendation").getOrCreate()

# 2. 데이터 로드 (Unity Catalog 테이블에서 불러오도록 변경)
catalog = "1dt_team8_databricks"
schema = "`final`"
path = f"{catalog}.{schema}"

try:
    # Unity Catalog에서 train, validation, test 테이블을 Spark DataFrame으로 직접 읽어옵니다.
    train_df_spark = spark.read.table(f"{path}.train_df")
    validation_df_spark = spark.read.table(f"{path}.validation_df")
    test_df_spark = spark.read.table(f"{path}.test_df")

    # movies_df_spark와 ratings_df_spark는 통합된 데이터에서 추출
    # movies_df_spark는 train_df에서 고유한 movieId, title, genres를 추출
    movies_df_spark = train_df_spark.select("movieId", "title", "genres").distinct()
    # ratings_df_spark는 train_df의 userId, movieId, rating, timestamp를 사용
    ratings_df_spark = train_df_spark.select("userId", "movieId", "rating", "timestamp") # 전체 평점 데이터 (train_df 기준)

    print("데이터를 Unity Catalog 테이블에서 성공적으로 불러왔습니다.")
    print(f"Train DataFrame 스키마: {train_df_spark.printSchema()}")
    print(f"Validation DataFrame 스키마: {validation_df_spark.printSchema()}")
    print(f"Test DataFrame 스키마: {test_df_spark.printSchema()}")

except Exception as e:
    print(f"Unity Catalog 테이블 로드 중 오류 발생: {e}")
    print("기존 경로 변수 방식으로 시도합니다. (데이터 경로를 다시 확인해주세요)")
    movies_path = "/databricks-datasets/movielens/small/movies.csv"
    ratings_path = "/databricks-datasets/movielens/small/ratings.csv"
    movies_df_spark = spark.read.csv(movies_path, header=True, inferSchema=True)
    ratings_df_spark = spark.read.csv(ratings_path, header=True, inferSchema=True)
    # 기존 movielens-small 데이터셋을 사용하는 경우 train/test 분할은 다시 진행해야 함.
    # 여기서는 Unity Catalog의 train/test/validation 분할된 데이터를 우선적으로 사용합니다.
    print("WARNING: Fallback to /databricks-datasets. Evaluation may not work as expected without pre-split data.")

# Pandas DataFrame으로 변환 (콘텐츠 기반 추천을 위해 TF-IDF를 사용하기 위함)
movies_df = movies_df_spark.toPandas()
ratings_df = ratings_df_spark.toPandas() # train_df_spark의 rating/timestamp 데이터 사용

# 추가: 평가를 위한 test_ratings_df (Pandas DataFrame) 생성
test_ratings_df = test_df_spark.toPandas()

# 데이터 확인 (주석 처리됨)
# print("Movies DataFrame head:")
# print(movies_df.head())
# print("\nRatings DataFrame head:")
# print(ratings_df.head())
# print("\nTest Ratings DataFrame head:")
# print(test_ratings_df.head())

# 3. 데이터 전처리 및 콘텐츠 기반 추천 시스템 구축

# 장르 데이터 클리닝 및 TF-IDF 벡터화
movies_df['genres'] = movies_df['genres'].fillna('')
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(movies_df['genres'])

print(f"\nTF-IDF matrix shape: {tfidf_matrix.shape}")

# 코사인 유사도 계산 (콘텐츠 기반 유사도)
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# 영화 제목과 인덱스를 매핑하는 시리즈 생성
indices = pd.Series(movies_df.index, index=movies_df['title']).drop_duplicates()

# 영화 ID로 제목을 찾는 헬퍼 함수 (Pandas DataFrame 사용)
def get_title_from_id(movie_id):
    """
    주어진 영화 ID에 해당하는 영화 제목을 movies_df에서 찾아 반환합니다.
    """
    if movie_id in movies_df['movieId'].values:
        return movies_df[movies_df['movieId'] == movie_id]['title'].iloc[0]
    return None

# 영화 이름으로 ID를 찾는 헬퍼 함수 (Pandas DataFrame 사용)
def get_movie_ids_from_names(movie_names):
    """
    주어진 영화 이름 리스트에 해당하는 영화 ID들을 찾아 반환합니다.
    찾을 수 없는 영화 이름도 별도로 반환합니다.
    """
    found_ids = []
    not_found_names = []
    for name in movie_names:
        match = movies_df[movies_df['title'].str.lower() == name.lower()]
        if not match.empty:
            found_ids.append(match['movieId'].iloc[0])
        else:
            not_found_names.append(name)
    return found_ids, not_found_names

# 4. 추천 함수 정의

def get_content_based_recommendations(movie_id, cosine_sim_matrix, movies_df_local, indices_local, num_recommendations=10):
    """
    콘텐츠(장르) 기반으로 유사한 영화를 추천합니다. (movie_id 기반)

    Args:
        movie_id (int): 기준이 될 영화의 ID.
        cosine_sim_matrix (np.array): 영화 간 코사인 유사도 행렬.
        movies_df_local (pd.DataFrame): 영화 정보가 담긴 Pandas DataFrame.
        indices_local (pd.Series): 영화 제목과 인덱스를 매핑하는 Series.
        num_recommendations (int): 추천할 영화의 개수.

    Returns:
        pd.DataFrame: 추천된 영화들의 정보 (Pandas DataFrame).
    """
    movie_title = get_title_from_id(movie_id)
    if not movie_title or movie_title not in indices_local:
        return pd.DataFrame()

    idx = indices_local[movie_title]
    sim_scores = list(enumerate(cosine_sim_matrix[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    sim_scores = sim_scores[1:num_recommendations + 1]
    movie_indices = [i[0] for i in sim_scores]
    return movies_df_local.iloc[movie_indices]


def recommend_movies_by_input(user_movie_ids, num_recommendations_total, num_popular_or_high_rated_fixed):
    """
    사용자가 입력한 영화 ID를 기반으로 콘텐츠 기반 추천 + 인기/고평점 영화를 섞어서 추천합니다.
    num_popular_or_high_rated_fixed 만큼 리뷰 많고 평점 높은 영화를 무조건 포함합니다.

    Args:
        user_movie_ids (list): 사용자가 입력한 영화 ID 리스트.
        num_recommendations_total (int): 최종적으로 추천할 총 영화 개수.
        num_popular_or_high_rated_fixed (int): 추천 목록에 무조건 포함할 인기/고평점 영화의 개수.

    Returns:
        pyspark.sql.DataFrame: 최종 추천된 영화들의 Spark DataFrame (movieId만 포함).
    """
    recommended_movies_df_pandas = pd.DataFrame(columns=['movieId']) # 추천된 영화 ID들을 저장할 Pandas DataFrame
    seen_movie_ids = set(user_movie_ids) # 사용자가 이미 입력한 영화 ID를 저장하여 추천 목록에서 제외

    # --- 1. 리뷰 많고 평점 높은 영화 목록 미리 계산 (Spark DataFrame 사용) ---
    # ratings_df_spark를 사용하여 영화별 리뷰 수(num_ratings)와 평균 평점(avg_rating)을 집계합니다.
    movie_stats_df_spark = ratings_df_spark.groupBy("movieId").agg(
        count(col("rating")).alias("num_ratings"),
        avg(col("rating")).alias("avg_rating")
    )

    min_ratings_threshold = 50 # 최소 리뷰 수 임계값
    popular_movies_spark_filtered = movie_stats_df_spark.filter(col("num_ratings") >= min_ratings_threshold) \
                                                         .orderBy(desc("avg_rating"), desc("num_ratings"))
    popular_movies_df_pandas = popular_movies_spark_filtered.toPandas() # Pandas DataFrame으로 변환

    # --- 2. 리뷰 많고 평점 높은 영화를 지정된 개수만큼 무조건 포함 ---
    popular_added_count = 0
    for _, row in popular_movies_df_pandas.iterrows():
        if row['movieId'] not in seen_movie_ids:
            recommended_movies_df_pandas = pd.concat([recommended_movies_df_pandas, pd.DataFrame([{'movieId': row['movieId']}])], ignore_index=True)
            seen_movie_ids.add(row['movieId'])
            popular_added_count += 1
            if popular_added_count >= num_popular_or_high_rated_fixed:
                break
    
    # --- 3. 나머지 추천 개수 (총 num_recommendations_total 중 인기영화 제외) ---
    remaining_recommendations_slots = num_recommendations_total - popular_added_count
    
    # 남은 슬롯이 없거나 음수가 되면 더 이상 콘텐츠 기반 추천을 할 필요 없음
    if remaining_recommendations_slots <= 0:
        final_recommendations = recommended_movies_df_pandas[~recommended_movies_df_pandas['movieId'].isin(user_movie_ids)] \
                                 .drop_duplicates(subset=['movieId']) \
                                 .head(num_recommendations_total) \
                                 .reset_index(drop=True)
        # 결과 스키마 정의 (movieId만 포함)
        schema = StructType([StructField("movieId", IntegerType(), True)])
        return spark.createDataFrame(final_recommendations, schema=schema)


    # --- 4. 콘텐츠 기반 추천으로 나머지 채우기 ---
    for movie_id in user_movie_ids:
        content_recs_for_one_movie = get_content_based_recommendations(
            movie_id, cosine_sim, movies_df, indices, remaining_recommendations_slots * 2
        )
        for _, row in content_recs_for_one_movie.iterrows():
            if row['movieId'] not in seen_movie_ids:
                recommended_movies_df_pandas = pd.concat([recommended_movies_df_pandas, pd.DataFrame([{'movieId': row['movieId']}])], ignore_index=True)
                seen_movie_ids.add(row['movieId'])
                if len(recommended_movies_df_pandas) - popular_added_count >= remaining_recommendations_slots:
                    break
        if len(recommended_movies_df_pandas) - popular_added_count >= remaining_recommendations_slots:
            break

    # --- 5. 여전히 추천 개수가 부족하면 추가 인기/고평점 영화로 채우기 ---
    if len(recommended_movies_df_pandas) < num_recommendations_total:
        additional_popular_needed = num_recommendations_total - len(recommended_movies_df_pandas)
        added_count = 0
        for _, row in popular_movies_df_pandas.iterrows():
            if row['movieId'] not in seen_movie_ids:
                recommended_movies_df_pandas = pd.concat([recommended_movies_df_pandas, pd.DataFrame([{'movieId': row['movieId']}])], ignore_index=True)
                seen_movie_ids.add(row['movieId'])
                added_count += 1
                if added_count >= additional_popular_needed:
                    break

    # 최종 추천 목록에서 필요한 개수만큼 자르기 (중복 및 입력 영화 제거 후)
    final_recommendations = recommended_movies_df_pandas[~recommended_movies_df_pandas['movieId'].isin(user_movie_ids)] \
                             .drop_duplicates(subset=['movieId']) \
                             .head(num_recommendations_total) \
                             .reset_index(drop=True)
    
    # 결과 스키마 정의 (movieId만 포함)
    schema = StructType([StructField("movieId", IntegerType(), True)])
    return spark.createDataFrame(final_recommendations, schema=schema)

# 5. 모델 테스트 (사용자 입력) - 주석 처리
# num_recommendations_output = 8
# num_guaranteed_popular = 2

# while True:
#     user_input_str = input("추천받고 싶은 영화 이름 또는 ID를 쉼표로 구분하여 3~5개 입력하세요 (예: Toy Story, Jumanji, Braveheart 또는 1,2,13): ")
    
#     is_id_input = True
#     processed_inputs = []
#     for item in user_input_str.split(','):
#         stripped_item = item.strip()
#         try:
#             processed_inputs.append(int(stripped_item))
#         except ValueError:
#             is_id_input = False
#             processed_inputs.append(stripped_item)

#     if not (3 <= len(processed_inputs) <= 5):
#         print("영화는 3개에서 5개 사이로 입력해야 합니다. 다시 입력해주세요.")
#         continue

#     input_movie_ids = []
#     input_movie_names_display = []

#     if is_id_input:
#         input_movie_ids = processed_inputs
#         found_movie_info = movies_df_spark.filter(col("movieId").isin(input_movie_ids)).select("movieId", "title").collect()
#         found_ids_set = {row.movieId for row in found_movie_info}
        
#         if len(found_ids_set) != len(input_movie_ids):
#             missing_ids = set(input_movie_ids) - found_ids_set
#             print(f"오류: 다음 영화 ID를 찾을 수 없습니다: {list(missing_ids)}. 다시 입력해주세요.")
#             continue
        
#         movie_id_to_title = {row.movieId: row.title for row in found_movie_info}
#         input_movie_names_display = [movie_id_to_title[mid] for mid in input_movie_ids]
#         print(f"\n입력된 영화 ID: {input_movie_ids}")

#     else:
#         input_movie_names = processed_inputs
#         input_movie_ids, not_found_names = get_movie_ids_from_names(input_movie_names)
        
#         if not_found_names:
#             print(f"다음 영화를 찾을 수 없거나 모호하여 추천을 진행할 수 없습니다: {', '.join(not_found_names)}")
#             continue
#         elif len(input_movie_ids) != len(input_movie_names):
#             print("입력된 모든 영화 이름을 영화 ID로 변환하지 못했습니다. 다시 시도해주세요.")
#             continue
        
#         input_movie_names_display = input_movie_names

#     break

# print(f"\n입력된 영화: {input_movie_names_display} (ID: {input_movie_ids})를 기반으로 영화를 추천합니다:")
# recommended_df = recommend_movies_by_input(input_movie_ids, num_recommendations_output, num_guaranteed_popular)
# # recommended_df는 movieId만 포함하므로, display 함수로 보여주기 위해 다시 join 필요
# final_display_df = recommended_df.join(movies_df_spark, "movieId", "left").select('title', 'genres')
# display(final_display_df)


# 6. 모델 성능 평가 및 시각화

# NDCG (Normalized Discounted Cumulative Gain) 계산 함수
def calculate_ndcg_at_k(recommended_items, relevant_items, k):
    """
    NDCG@k를 계산합니다.
    Args:
        recommended_items (list): 추천된 아이템 ID 리스트.
        relevant_items (set): 관련성 있는 아이템 ID 집합.
        k (int): 평가할 상위 K개 아이템.
    Returns:
        float: NDCG@k 값.
    """
    if not relevant_items:
        return 0.0

    dcg = 0.0
    idcg = 0.0

    for i in range(k):
        if i < len(recommended_items):
            item = recommended_items[i]
            if item in relevant_items:
                dcg += 1.0 / np.log2(i + 2) # i+1은 0-based index이므로, log2(position + 1)
        
        # Ideal DCG (가장 이상적인 추천 순서)
        if i < len(relevant_items):
            idcg += 1.0 / np.log2(i + 2)

    return dcg / idcg if idcg > 0 else 0.0

# 평가 지표 계산 함수 (Precision@k, Recall@k, NDCG@k)
def evaluate_recommender(model_func, movies_df_eval, ratings_df_eval, indices_eval, cosine_sim_matrix_eval,
                         test_ratings_df_eval, train_ratings_df_eval, k=10, num_popular_fixed=1):
    """
    추천 시스템의 평균 Precision@k, Recall@k, NDCG@k를 계산하고,
    각 유저별 추천 결과를 포함하는 DataFrame을 반환합니다.

    Args:
        model_func (function): 영화 추천을 수행하는 함수 (여기서는 recommend_movies_by_input).
        movies_df_eval (pd.DataFrame): 영화 정보 DataFrame.
        ratings_df_eval (pd.DataFrame): 전체 평점 정보 DataFrame (훈련 데이터를 포함).
        indices_eval (pd.Series): 영화 제목과 인덱스 매핑 Series.
        cosine_sim_matrix_eval (np.array): 코사인 유사도 행렬.
        test_ratings_df_eval (pd.DataFrame): 테스트용 평점 DataFrame.
        train_ratings_df_eval (pd.DataFrame): 훈련용 평점 DataFrame.
        k (int): 평가할 상위 K개 영화.
        num_popular_fixed (int): 모델이 고정적으로 포함할 인기 영화 개수.
    Returns:
        tuple: (평균 Precision@k, 평균 Recall@k, 평균 NDCG@k, 평가된 사용자 수, 사용자별 추천 결과 DataFrame)
    """
    total_precision = 0
    total_recall = 0
    total_ndcg = 0
    num_users_evaluated = 0
    
    # 사용자별 추천 결과를 저장할 리스트
    user_recommendations_data = []

    # 테스트 데이터셋에서 임의의 사용자 샘플링 (최대 100명 또는 고유 사용자 수 중 더 작은 값)
    sample_users = test_ratings_df_eval['userId'].sample(min(100, test_ratings_df_eval['userId'].nunique()), random_state=42).unique()

    for user_id in sample_users:
        # 해당 사용자의 훈련 데이터에서 가장 높은 평점을 준 영화들을 '입력 영화'로 가정
        # (4점 이상 평점을 준 영화들 중 상위 3개)
        user_train_movies_high_rated = train_ratings_df_eval[(train_ratings_df_eval['userId'] == user_id) & (train_ratings_df_eval['rating'] >= 4)] \
                                        .sort_values(by='rating', ascending=False)
        
        if len(user_train_movies_high_rated) < 3: # 최소 3개 이상의 높은 평점 영화가 없으면 평가 제외
            continue
            
        seed_movie_ids = user_train_movies_high_rated.head(3)['movieId'].tolist()

        if not seed_movie_ids:
            continue

        # 추천 받기
        recommended_movies_df_spark = model_func(
            seed_movie_ids, k, num_popular_or_high_rated_fixed=num_popular_fixed
        )
        recommended_movie_ids_list = [row.movieId for row in recommended_movies_df_spark.collect()]
        
        if not recommended_movie_ids_list:
            continue

        # 실제 관련성 있는 영화: 테스트 데이터에서 해당 사용자가 4점 이상 평점을 준 영화
        actual_relevant_movies_df = test_ratings_df_eval[(test_ratings_df_eval['userId'] == user_id) & (test_ratings_df_eval['rating'] >= 4)]
        actual_relevant_movie_ids = set(actual_relevant_movies_df['movieId'].tolist())

        # 추천된 영화 ID 리스트를 k개로 자르기
        recommended_k = recommended_movie_ids_list[:k]

        # Precision@k
        true_positives = len(set(recommended_k).intersection(actual_relevant_movie_ids))
        precision_k = true_positives / k if k > 0 else 0

        # Recall@k
        recall_k = true_positives / len(actual_relevant_movie_ids) if len(actual_relevant_movie_ids) > 0 else 0

        # NDCG@k
        ndcg_k = calculate_ndcg_at_k(recommended_k, actual_relevant_movie_ids, k)

        total_precision += precision_k
        total_recall += recall_k
        total_ndcg += ndcg_k
        num_users_evaluated += 1

        # 사용자별 추천 결과 저장
        user_recommendations_data.append({
            'userId': user_id,
            'seed_movie_ids': seed_movie_ids,
            'recommended_movie_ids': recommended_k,
            'actual_relevant_movie_ids': list(actual_relevant_movie_ids), # Set to list for DataFrame
            'precision_at_k': precision_k,
            'recall_at_k': recall_k,
            'ndcg_at_k': ndcg_k
        })

    user_recs_df = pd.DataFrame(user_recommendations_data)


    if num_users_evaluated == 0:
        return 0, 0, 0, 0, pd.DataFrame()

    avg_precision = total_precision / num_users_evaluated
    avg_recall = total_recall / num_users_evaluated
    avg_ndcg = total_ndcg / num_users_evaluated

    return avg_precision, avg_recall, avg_ndcg, num_users_evaluated, user_recs_df

# 모델 성능 평가 함수 호출
def model_performance_evaluation(k_val=10, popular_count=2):
    print("\n모델 성능 평가를 시작합니다...")
    # evaluate_recommender 함수에 train_ratings_df_eval를 전달 (훈련 데이터)
    avg_precision, avg_recall, avg_ndcg, num_evaluated_users, user_recs_df = evaluate_recommender(
        recommend_movies_by_input, movies_df, ratings_df, indices, cosine_sim, 
        test_ratings_df, ratings_df, # ratings_df는 train_df_spark.toPandas()로 생성되었으므로, 훈련 데이터 역할을 합니다.
        k=k_val, # 평가할 K 값
        num_popular_fixed=popular_count # 포함될 인기 영화 개수
    )

    print(f"\n평가된 사용자 수: {num_evaluated_users}")
    print(f"평균 Precision@{k_val}: {avg_precision:.4f}")
    print(f"평균 Recall@{k_val}: {avg_recall:.4f}")
    print(f"평균 NDCG@{k_val}: {avg_ndcg:.4f}")

    # 성능 시각화
    metrics = [f'Precision@{k_val}', f'Recall@{k_val}', f'NDCG@{k_val}']
    values = [avg_precision, avg_recall, avg_ndcg]

    plt.figure(figsize=(8, 6))
    sns.barplot(x=metrics, y=values, palette='viridis')
    plt.title(f'Recommendation System Performance Metrics (k={k_val})')
    plt.ylabel('Score')
    plt.ylim(0, 1)
    for i, v in enumerate(values):
        plt.text(i, v + 0.02, f"{v:.2f}", ha='center', va='bottom', fontsize=10)
    plt.show()

    # 사용자별 추천 결과 DataFrame 출력
    print("\n--- 사용자별 추천 결과 (샘플) ---")
    # recommended_movie_ids를 영화 제목으로 변환하여 보기 좋게 출력
    user_recs_df['recommended_movie_titles'] = user_recs_df['recommended_movie_ids'].apply(
        lambda ids: [get_title_from_id(mid) for mid in ids]
    )
    user_recs_df['seed_movie_titles'] = user_recs_df['seed_movie_ids'].apply(
        lambda ids: [get_title_from_id(mid) for mid in ids]
    )
    user_recs_df['actual_relevant_movie_titles'] = user_recs_df['actual_relevant_movie_ids'].apply(
        lambda ids: [get_title_from_id(mid) for mid in ids]
    )

    # 필요한 컬럼만 선택하여 출력
    display(user_recs_df[['userId', 'recommended_movie_titles']])

# 평가 실행 (예: k=10, 인기 영화 2개 고정)
model_performance_evaluation(k_val=10, popular_count=2)

In [0]:
# Microsoft Azure Databricks Notebook.
# Content-Based + 인기 영화 추천 시스템 (Hybrid - Multi-Hot Encoding & Weighted Linear Combination)

# 1. 라이브러리 임포트
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer # 텍스트 데이터를 TF-IDF 특징 벡터로 변환
from sklearn.metrics.pairwise import linear_kernel # 코사인 유사도(선형 커널)를 효율적으로 계산
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, count, avg, desc, udf
from pyspark.sql.types import ArrayType, StringType, IntegerType, StructType, StructField, DoubleType

# Spark 세션 초기화 (Databricks에서는 자동으로 초기화되지만 명시적으로 정의 가능)
spark = SparkSession.builder.appName("MovieRecommendation").getOrCreate()

# 2. 데이터 로드 (Unity Catalog 테이블에서 불러오도록 변경)
catalog = "1dt_team8_databricks"
schema = "`final`"
path = f"{catalog}.{schema}"

try:
    # Unity Catalog에서 train, validation, test 테이블을 Spark DataFrame으로 직접 읽어옵니다.
    train_df_spark = spark.read.table(f"{path}.train_df")
    validation_df_spark = spark.read.table(f"{path}.validation_df")
    test_df_spark = spark.read.table(f"{path}.test_df")

    # movies_df_spark와 ratings_df_spark는 통합된 데이터에서 추출
    # movies_df_spark는 train_df에서 고유한 movieId, title, genres를 추출
    movies_df_spark = train_df_spark.select("movieId", "title", "genres").distinct()
    # ratings_df_spark는 train_df의 userId, movieId, rating, timestamp를 사용
    ratings_df_spark = train_df_spark.select("userId", "movieId", "rating", "timestamp") # 전체 평점 데이터 (train_df 기준)

    print("데이터를 Unity Catalog 테이블에서 성공적으로 불러왔습니다.")
    print(f"Train DataFrame 스키마: {train_df_spark.printSchema()}")
    print(f"Validation DataFrame 스키마: {validation_df_spark.printSchema()}")
    print(f"Test DataFrame 스키마: {test_df_spark.printSchema()}")

except Exception as e:
    print(f"Unity Catalog 테이블 로드 중 오류 발생: {e}")
    print("기존 경로 변수 방식으로 시도합니다. (데이터 경로를 다시 확인해주세요)")
    movies_path = "/databricks-datasets/movielens/small/movies.csv"
    ratings_path = "/databricks-datasets/movielens/small/ratings.csv"
    movies_df_spark = spark.read.csv(movies_path, header=True, inferSchema=True)
    ratings_df_spark = spark.read.csv(ratings_path, header=True, inferSchema=True)
    print("WARNING: Fallback to /databricks-datasets. Evaluation may not work as expected without pre-split data.")

# Pandas DataFrame으로 변환
movies_df = movies_df_spark.toPandas()
ratings_df = ratings_df_spark.toPandas()
test_ratings_df = test_df_spark.toPandas()

# 3. 데이터 전처리 및 콘텐츠 기반 추천 시스템 구축 (Multi-Hot Encoding 적용)

# 장르 데이터 클리닝 및 멀티-핫 인코딩
movies_df['genres'] = movies_df['genres'].fillna('')

# 각 영화의 장르를 '|' 기준으로 분리하고, 전체 장르 집합을 얻음
all_genres = set()
for genres_str in movies_df['genres']:
    for genre in genres_str.split('|'):
        if genre: # 빈 문자열 방지
            all_genres.add(genre)

# 각 장르를 공백으로 구분된 문자열로 변환 (TF-IDF를 위해)
# 예: 'Action|Adventure' -> 'Action Adventure'
movies_df['genres_processed'] = movies_df['genres'].apply(lambda x: ' '.join(x.split('|')) if x else '')

tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(movies_df['genres_processed'])

print(f"\nTF-IDF matrix shape (after Multi-Hot Encoding for genres): {tfidf_matrix.shape}")

# 코사인 유사도 계산 (콘텐츠 기반 유사도)
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# 영화 제목과 인덱스를 매핑하는 시리즈 생성
indices = pd.Series(movies_df.index, index=movies_df['title']).drop_duplicates()

# 영화 ID로 제목을 찾는 헬퍼 함수 (Pandas DataFrame 사용)
def get_title_from_id(movie_id):
    """
    주어진 영화 ID에 해당하는 영화 제목을 movies_df에서 찾아 반환합니다.
    """
    if movie_id in movies_df['movieId'].values:
        return movies_df[movies_df['movieId'] == movie_id]['title'].iloc[0]
    return None

# 영화 이름으로 ID를 찾는 헬퍼 함수 (Pandas DataFrame 사용)
def get_movie_ids_from_names(movie_names):
    """
    주어진 영화 이름 리스트에 해당하는 영화 ID들을 찾아 반환합니다.
    찾을 수 없는 영화 이름도 별도로 반환합니다.
    """
    found_ids = []
    not_found_names = []
    for name in movie_names:
        match = movies_df[movies_df['title'].str.lower() == name.lower()]
        if not match.empty:
            found_ids.append(match['movieId'].iloc[0])
        else:
            not_found_names.append(name)
    return found_ids, not_found_names

# 4. 추천 함수 정의 (가중치 기반 선형 결합 적용)

def get_content_based_recommendations(movie_id, cosine_sim_matrix, movies_df_local, indices_local, num_recommendations=10):
    """
    콘텐츠(장르) 기반으로 유사한 영화를 추천합니다. (movie_id 기반)

    Args:
        movie_id (int): 기준이 될 영화의 ID.
        cosine_sim_matrix (np.array): 영화 간 코사인 유사도 행렬.
        movies_df_local (pd.DataFrame): 영화 정보가 담긴 Pandas DataFrame.
        indices_local (pd.Series): 영화 제목과 인덱스를 매핑하는 Series.
        num_recommendations (int): 추천할 영화의 개수.

    Returns:
        pd.DataFrame: 추천된 영화들의 정보 (Pandas DataFrame).
    """
    movie_title = get_title_from_id(movie_id)
    if not movie_title or movie_title not in indices_local:
        return pd.DataFrame()

    idx = indices_local[movie_title]
    sim_scores = list(enumerate(cosine_sim_matrix[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    sim_scores = sim_scores[1:num_recommendations + 1] # 자기 자신 제외
    movie_indices = [i[0] for i in sim_scores]
    return movies_df_local.iloc[movie_indices]


def recommend_movies_weighted_hybrid(user_movie_ids, num_recommendations_total,
                                      content_weight=0.7, popular_weight=0.3):
    """
    사용자가 입력한 영화 ID를 기반으로 콘텐츠 기반 추천과 인기/고평점 영화 추천을
    가중치를 주어 선형 결합하여 추천합니다.

    Args:
        user_movie_ids (list): 사용자가 입력한 영화 ID 리스트.
        num_recommendations_total (int): 최종적으로 추천할 총 영화 개수.
        content_weight (float): 콘텐츠 기반 추천에 부여할 가중치 (0.0 ~ 1.0).
        popular_weight (float): 인기/고평점 추천에 부여할 가중치 (0.0 ~ 1.0).
                                (content_weight + popular_weight는 1.0이 될 필요는 없지만, 일반적으로는 1.0으로 정규화하여 사용)

    Returns:
        pyspark.sql.DataFrame: 최종 추천된 영화들의 Spark DataFrame (movieId만 포함).
    """
    if not user_movie_ids:
        # 입력 영화가 없으면 인기/고평점 영화만 추천
        popular_movies_spark_filtered = ratings_df_spark.groupBy("movieId").agg(
            count(col("rating")).alias("num_ratings"),
            avg(col("rating")).alias("avg_rating")
        ).filter(col("num_ratings") >= 50) \
         .orderBy(desc("avg_rating"), desc("num_ratings"))

        final_recommendations_spark = popular_movies_spark_filtered.select("movieId").limit(num_recommendations_total)
        return final_recommendations_spark


    seen_movie_ids = set(user_movie_ids)

    # --- 1. 콘텐츠 기반 추천 목록 생성 ---
    content_based_recs_df = pd.DataFrame(columns=['movieId', 'score'])
    for movie_id in user_movie_ids:
        # 각 입력 영화에 대해 충분히 많은 콘텐츠 기반 추천을 가져옴
        content_recs_for_one_movie = get_content_based_recommendations(
            movie_id, cosine_sim, movies_df, indices, num_recommendations_total * 5 # 충분히 많이 가져옴
        )
        if not content_recs_for_one_movie.empty:
            # 코사인 유사도 점수를 score로 사용 (0~1 범위)
            current_movie_idx = movies_df[movies_df['movieId'] == movie_id].index[0]
            sim_scores_for_movie = linear_kernel(tfidf_matrix[current_movie_idx], tfidf_matrix[content_recs_for_one_movie.index]).flatten()

            temp_df = pd.DataFrame({
                'movieId': content_recs_for_one_movie['movieId'],
                'score': sim_scores_for_movie
            })
            content_based_recs_df = pd.concat([content_based_recs_df, temp_df], ignore_index=True)

    # 중복 제거 및 점수 합산 (동일 영화에 대한 여러 입력 영화로부터의 추천을 합산)
    content_based_recs_df = content_based_recs_df.groupby('movieId')['score'].sum().reset_index()
    # 사용자가 본 영화 제외
    content_based_recs_df = content_based_recs_df[~content_based_recs_df['movieId'].isin(seen_movie_ids)]
    # 점수 정규화 (0~1)
    if not content_based_recs_df.empty:
        max_content_score = content_based_recs_df['score'].max()
        if max_content_score > 0:
            content_based_recs_df['score'] = content_based_recs_df['score'] / max_content_score
    content_based_recs_df = content_based_recs_df.rename(columns={'score': 'content_score'})


    # --- 2. 인기/고평점 영화 목록 생성 ---
    movie_stats_df_spark = ratings_df_spark.groupBy("movieId").agg(
        count(col("rating")).alias("num_ratings"),
        avg(col("rating")).alias("avg_rating")
    )
    min_ratings_threshold = 50 # 최소 리뷰 수 임계값
    popular_movies_spark_filtered = movie_stats_df_spark.filter(col("num_ratings") >= min_ratings_threshold) \
                                                          .orderBy(desc("avg_rating"), desc("num_ratings"))
    popular_movies_df_pandas = popular_movies_spark_filtered.toPandas()

    # 인기/고평점 점수 정규화 (avg_rating을 사용, 0~1)
    if not popular_movies_df_pandas.empty:
        max_avg_rating = popular_movies_df_pandas['avg_rating'].max()
        if max_avg_rating > 0:
            popular_movies_df_pandas['popular_score'] = popular_movies_df_pandas['avg_rating'] / max_avg_rating
        else: # max_avg_rating이 0인 경우 (예외 처리)
             popular_movies_df_pandas['popular_score'] = 0.0
    else:
        popular_movies_df_pandas['popular_score'] = 0.0


    # --- 3. 두 추천 목록 결합 및 가중치 적용 ---
    # 각 영화에 대한 콘텐츠 점수와 인기 점수를 합칠 DataFrame 준비
    # 먼저, 모든 영화 ID를 포함하는 기준 DataFrame 생성
    all_movie_ids = movies_df_spark.select("movieId").distinct().toPandas()
    hybrid_recs_df = all_movie_ids.copy()
    hybrid_recs_df['content_score'] = 0.0
    hybrid_recs_df['popular_score'] = 0.0

    # 콘텐츠 점수 병합
    hybrid_recs_df = pd.merge(hybrid_recs_df, content_based_recs_df, on='movieId', how='left', suffixes=('_old', ''))
    hybrid_recs_df['content_score'] = hybrid_recs_df['content_score'].fillna(0) # merge 시 NaN 처리
    # popular_score도 초기화 후 병합
    hybrid_recs_df = pd.merge(hybrid_recs_df, popular_movies_df_pandas[['movieId', 'popular_score']], on='movieId', how='left', suffixes=('_old', ''))
    hybrid_recs_df['popular_score'] = hybrid_recs_df['popular_score'].fillna(0) # merge 시 NaN 처리

    # 최종 가중치 점수 계산
    hybrid_recs_df['final_score'] = (hybrid_recs_df['content_score'] * content_weight) + \
                                    (hybrid_recs_df['popular_score'] * popular_weight)

    # 사용자가 이미 본 영화는 추천에서 제외
    hybrid_recs_df = hybrid_recs_df[~hybrid_recs_df['movieId'].isin(seen_movie_ids)]

    # 최종 점수를 기준으로 정렬하고 상위 N개 추천
    final_recommendations_pandas = hybrid_recs_df.sort_values(by='final_score', ascending=False) \
                                                 .drop_duplicates(subset=['movieId']) \
                                                 .head(num_recommendations_total) \
                                                 .reset_index(drop=True)

    # Spark DataFrame으로 변환
    schema = StructType([StructField("movieId", IntegerType(), True),
                         StructField("content_score", DoubleType(), True),
                         StructField("popular_score", DoubleType(), True),
                         StructField("final_score", DoubleType(), True)]) # 점수 컬럼도 포함하여 반환 가능
    # 다만 평가 함수는 movieId만 필요하므로, 여기서는 movieId만 포함된 DF로 반환
    final_recommendations_spark = spark.createDataFrame(final_recommendations_pandas[['movieId']], StructType([StructField("movieId", IntegerType(), True)]))

    return final_recommendations_spark

# 5. 모델 테스트 (사용자 입력) - 주석 처리
# 예제 사용:
# num_recommendations_output = 10
# content_w = 0.7
# popular_w = 0.3

# # 테스트할 입력 영화 ID (예: Toy Story, Jumanji, Grumpier Old Men)
# # movies_df.head()를 보고 실제 존재하는 영화 ID를 입력하세요.
# example_user_movie_ids = [1, 2, 3] # 예시 ID

# print(f"\n입력된 영화: {[get_title_from_id(mid) for mid in example_user_movie_ids]} (ID: {example_user_movie_ids})를 기반으로 영화를 추천합니다:")
# recommended_df = recommend_movies_weighted_hybrid(example_user_movie_ids, num_recommendations_output,
#                                                   content_weight=content_w, popular_weight=popular_w)

# # recommended_df는 movieId만 포함하므로, display 함수로 보여주기 위해 다시 join 필요
# final_display_df = recommended_df.join(movies_df_spark, "movieId", "left").select('title', 'genres')
# print("\n--- 추천 결과 ---")
# display(final_display_df)


# 6. 모델 성능 평가 및 시각화

# NDCG (Normalized Discounted Cumulative Gain) 계산 함수
def calculate_ndcg_at_k(recommended_items, relevant_items, k):
    """
    NDCG@k를 계산합니다.
    Args:
        recommended_items (list): 추천된 아이템 ID 리스트.
        relevant_items (set): 관련성 있는 아이템 ID 집합.
        k (int): 평가할 상위 K개 아이템.
    Returns:
        float: NDCG@k 값.
    """
    if not relevant_items:
        return 0.0

    dcg = 0.0
    idcg = 0.0

    for i in range(k):
        if i < len(recommended_items):
            item = recommended_items[i]
            if item in relevant_items:
                dcg += 1.0 / np.log2(i + 2) # i+1은 0-based index이므로, log2(position + 1)
        
        # Ideal DCG (가장 이상적인 추천 순서)
        if i < len(relevant_items):
            idcg += 1.0 / np.log2(i + 2)

    return dcg / idcg if idcg > 0 else 0.0

# 평가 지표 계산 함수 (Precision@k, Recall@k, NDCG@k)
def evaluate_recommender(model_func, movies_df_eval, ratings_df_eval, indices_eval, cosine_sim_matrix_eval,
                         test_ratings_df_eval, train_ratings_df_eval, k=10, content_w=0.7, popular_w=0.3):
    """
    추천 시스템의 평균 Precision@k, Recall@k, NDCG@k를 계산하고,
    각 유저별 추천 결과를 포함하는 DataFrame을 반환합니다.

    Args:
        model_func (function): 영화 추천을 수행하는 함수 (여기서는 recommend_movies_weighted_hybrid).
        movies_df_eval (pd.DataFrame): 영화 정보 DataFrame.
        ratings_df_eval (pd.DataFrame): 전체 평점 정보 DataFrame (훈련 데이터를 포함).
        indices_eval (pd.Series): 영화 제목과 인덱스 매핑 Series.
        cosine_sim_matrix_eval (np.array): 코사인 유사도 행렬.
        test_ratings_df_eval (pd.DataFrame): 테스트용 평점 DataFrame.
        train_ratings_df_eval (pd.DataFrame): 훈련용 평점 DataFrame.
        k (int): 평가할 상위 K개 영화.
        content_w (float): 콘텐츠 기반 추천 가중치.
        popular_w (float): 인기/고평점 추천 가중치.
    Returns:
        tuple: (평균 Precision@k, 평균 Recall@k, 평균 NDCG@k, 평가된 사용자 수, 사용자별 추천 결과 DataFrame)
    """
    total_precision = 0
    total_recall = 0
    total_ndcg = 0
    num_users_evaluated = 0
    
    # 사용자별 추천 결과를 저장할 리스트
    user_recommendations_data = []

    # 테스트 데이터셋에서 임의의 사용자 샘플링 (최대 100명 또는 고유 사용자 수 중 더 작은 값)
    sample_users = test_ratings_df_eval['userId'].sample(min(100, test_ratings_df_eval['userId'].nunique()), random_state=42).unique()

    for user_id in sample_users:
        # 해당 사용자의 훈련 데이터에서 가장 높은 평점을 준 영화들을 '입력 영화'로 가정
        # (4점 이상 평점을 준 영화들 중 상위 3개)
        user_train_movies_high_rated = train_ratings_df_eval[(train_ratings_df_eval['userId'] == user_id) & (train_ratings_df_eval['rating'] >= 4)] \
                                            .sort_values(by='rating', ascending=False)
        
        if len(user_train_movies_high_rated) < 3: # 최소 3개 이상의 높은 평점 영화가 없으면 평가 제외
            continue
            
        seed_movie_ids = user_train_movies_high_rated.head(3)['movieId'].tolist()

        if not seed_movie_ids:
            continue

        # 추천 받기
        recommended_movies_df_spark = model_func(
            seed_movie_ids, k, content_weight=content_w, popular_weight=popular_w
        )
        recommended_movie_ids_list = [row.movieId for row in recommended_movies_df_spark.collect()]
        
        if not recommended_movie_ids_list:
            continue

        # 실제 관련성 있는 영화: 테스트 데이터에서 해당 사용자가 4점 이상 평점을 준 영화
        actual_relevant_movies_df = test_ratings_df_eval[(test_ratings_df_eval['userId'] == user_id) & (test_ratings_df_eval['rating'] >= 4)]
        actual_relevant_movie_ids = set(actual_relevant_movies_df['movieId'].tolist())

        # 추천된 영화 ID 리스트를 k개로 자르기
        recommended_k = recommended_movie_ids_list[:k]

        # Precision@k
        true_positives = len(set(recommended_k).intersection(actual_relevant_movie_ids))
        precision_k = true_positives / k if k > 0 else 0

        # Recall@k
        recall_k = true_positives / len(actual_relevant_movie_ids) if len(actual_relevant_movie_ids) > 0 else 0

        # NDCG@k
        ndcg_k = calculate_ndcg_at_k(recommended_k, actual_relevant_movie_ids, k)

        total_precision += precision_k
        total_recall += recall_k
        total_ndcg += ndcg_k
        num_users_evaluated += 1

        # 사용자별 추천 결과 저장
        user_recommendations_data.append({
            'userId': user_id,
            'seed_movie_ids': seed_movie_ids,
            'recommended_movie_ids': recommended_k,
            'actual_relevant_movie_ids': list(actual_relevant_movie_ids), # Set to list for DataFrame
            'precision_at_k': precision_k,
            'recall_at_k': recall_k,
            'ndcg_at_k': ndcg_k
        })

    user_recs_df = pd.DataFrame(user_recommendations_data)


    if num_users_evaluated == 0:
        return 0, 0, 0, 0, pd.DataFrame()

    avg_precision = total_precision / num_users_evaluated
    avg_recall = total_recall / num_users_evaluated
    avg_ndcg = total_ndcg / num_users_evaluated

    return avg_precision, avg_recall, avg_ndcg, num_users_evaluated, user_recs_df

# 모델 성능 평가 함수 호출
def model_performance_evaluation(k_val=10, content_weight=0.7, popular_weight=0.3):
    print("\n모델 성능 평가를 시작합니다...")
    # evaluate_recommender 함수에 train_ratings_df_eval를 전달 (훈련 데이터)
    avg_precision, avg_recall, avg_ndcg, num_evaluated_users, user_recs_df = evaluate_recommender(
        recommend_movies_weighted_hybrid, movies_df, ratings_df, indices, cosine_sim, 
        test_ratings_df, ratings_df, # ratings_df는 train_df_spark.toPandas()로 생성되었으므로, 훈련 데이터 역할을 합니다.
        k=k_val, # 평가할 K 값
        content_w=content_weight, popular_w=popular_weight # 가중치 파라미터 전달
    )

    print(f"\n평가된 사용자 수: {num_evaluated_users}")
    print(f"평균 Precision@{k_val}: {avg_precision:.4f}")
    print(f"평균 Recall@{k_val}: {avg_recall:.4f}")
    print(f"평균 NDCG@{k_val}: {avg_ndcg:.4f}")

    # 성능 시각화
    metrics = [f'Precision@{k_val}', f'Recall@{k_val}', f'NDCG@{k_val}']
    values = [avg_precision, avg_recall, avg_ndcg]

    plt.figure(figsize=(8, 6))
    sns.barplot(x=metrics, y=values, palette='viridis')
    plt.title(f'Recommendation System Performance Metrics (k={k_val}, Content Weight={content_weight}, Popular Weight={popular_weight})')
    plt.ylabel('Score')
    plt.ylim(0, 1)
    for i, v in enumerate(values):
        plt.text(i, v + 0.02, f"{v:.2f}", ha='center', va='bottom', fontsize=10)
    plt.show()

    # 사용자별 추천 결과 DataFrame 출력
    print("\n--- 사용자별 추천 결과 (샘플) ---")
    # recommended_movie_ids를 영화 제목으로 변환하여 보기 좋게 출력
    user_recs_df['recommended_movie_titles'] = user_recs_df['recommended_movie_ids'].apply(
        lambda ids: [get_title_from_id(mid) for mid in ids]
    )
    user_recs_df['seed_movie_titles'] = user_recs_df['seed_movie_ids'].apply(
        lambda ids: [get_title_from_id(mid) for mid in ids]
    )
    user_recs_df['actual_relevant_movie_titles'] = user_recs_df['actual_relevant_movie_ids'].apply(
        lambda ids: [get_title_from_id(mid) for mid in ids]
    )

    # 필요한 컬럼만 선택하여 출력
    display(user_recs_df[['userId', 'seed_movie_titles', 'recommended_movie_titles', 'precision_at_k', 'recall_at_k', 'ndcg_at_k']])

# 평가 실행 (예: k=10, 콘텐츠 가중치 0.7, 인기 가중치 0.3)
# 이 가중치 값들을 변경하면서 최적의 성능을 찾아보세요.
model_performance_evaluation(k_val=10, content_weight=0.5, popular_weight=0.5)